# Lab12-2 Hi Hello RNN
### 2018.09.30(일)
> RNN

Lab12-1에서는 Input 과 output을 확인했다면, 여기서는 RNN을 직접 학습시켜서 model을 만드는 코드를 짤 것.

__"hihello"라는 글자를 학습시키자.__<br>
즉, 우리가 h를 주면 그다음 글자가 무엇인지 맞추도록 하기.<br>
여기서 핵심은, 같은 h더라도 sequential 하게 어떤때는 i 가, 어떤 때는 e가 도출되어야 한다.<br>
이것은 이전까지의 Forward net으로는 구현하지 못하는 부분이었음. RNN이니까 가능.

![lec12_12](../../img/lec12_12.png)

## 구현의 기본적인 정의들.

![lec12_13](../../img/lec12_13.png)

우리는 여기서,
1. Input-dimension
2. Output-demension
3. Sequence-length
4. batch-size

를 정해주어야 한다.

1. 출현하는 글자가 5개 ( h, i, e, l, o ) 이므로, Input-dimension = 5
2. (h->i,i->h,h->e,e->l,l->l,l->o) 총 6번의 sequence이므로, Sequence-length= 6
3. Hidden, 즉, 출력의 dimension 은 (i,h,e,l,o)이므로, ouput-dimension = 5
4. 입력된 문자는 ( "hihello" )하나이므로, batch-size = 1 밖에 안됨

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

/Users/Choi-seonyeol/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 1. RNN Cell 을 만든다.
1. RNN model : rnn_cell = rnn_cell.BasicRNNCell(rnn_size)
2. LSTM model : rnn_cell = rnn_cell.BasicLSTMCell(rnn_size)
3. GRU model : rnn_cell = rnn_cell.GRUCell(rnn_size)

-> 여기서, rnn_size 는 htdden-rnn-size(output-dimension) 으로, "5"

## 2. Execute RNN
```
outputs, _states = tf.nn.dynamic_rnn(rnn_cell, X, initial_state=initial_state, dtype = tf.flaot32)
```
-> 여기서, X는 input-dimension은"5", 그리고 Sequence_length는 "6" 므로, 이에 맞게 X를 만들어서 입력

In [2]:
hidden_size = 5      # output from the LSTM
input_dim = 5        # one-hot size
batch_size = 1       # one sentence
sequence_length = 6  # |ihello| == 6

### 이를 기반으로 데이터 만들기
## Data creation

In [3]:
idx2char = ['h', 'i', 'e', 'l', 'o'] # h=0, i=1, e=2, l=3, o=4
# idx2char[0] -> "h" 가 나오도록 indexing
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell

# one_hot으로 치환된 것. 즉, x_one_hot[0] -> h인것.
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

# 우리가 학습하고싶은 TRUE 값 (label)
y_data = [[1, 0, 2, 3, 3, 4]]    # ihello


### Feed to  RNN

In [4]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) # X one-hot
# batch_size = None 인것은, 어챠피 한개라서 batch가 많아도 괜찮아! 라고 말해주는 것.
Y = tf.placeholder(tf.int32, [None, sequence_length]) # Y Label
# 결과는 sequence의 크기만큼 도출이 될것. -> sequence_length

cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
# hidden_size 는 num_units 가 된다.
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

### Cost: sequence_loss

In [5]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights = weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
# 여기서 주의해야할 것은, 사실 여기서는 loss계산에서 logits에 rnn의 output을 바로 넣었는데, 이는 좋지 못하다.
# rnn의 output을 통해 예측을 한 후 loss 계산이 더 옳다.

### Training

In [6]:
prediction = tf.argmax(outputs, axis = 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2001):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.640098 prediction:  [[3 3 3 4 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllooo
1 loss: 1.5719721 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.5155629 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.4554002 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
4 loss: 1.3955768 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
5 loss: 1.3389432 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
6 loss: 1.2822633 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
7 loss: 1.2256817 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
8 loss: 1.1756244 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllll
9 loss: 1.1470249 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predi

### 번외 / Cost: sequence_loss 의 예시
output이 얼마나 좋은가 알기 위해 loss를 계산.
sequence들을 알고 있을때, 각각 cross entropy를 계산하는 등, 방법은 많지만 아주 복잡할 것.
이를 위해,tensorflow에서는 sequence_loss라는 함수를 만들었음

In [ ]:
# [batch_size, sequence_length]
y_data = tf.constant([[1,1,1]])
# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)
# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

with tf.Session() as sess:
    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
    # logits 에 우리들의 예측을 넣음, 
    # targets에 sequence를 넣음(True), 
    # weight에다가는 각각의 자리를 얼마나 중요하게 생각하는지를 넣음(여기서는 일단 모두 1로)
    sess.run(tf.global_variables_initializer())
    print("Loss: ", sequence_loss.eval())

### 번외 / Cost: sequence_loss 의 비교 예시
이게 잘 되는지 한번 확인해보기 위해, prediction1과 prediction2를 만들어서 각각의 cost를 비교해보자.

그리고, cost가 왜 이렇게나오는지 생각도 해 보자.

In [ ]:
# [batch_size, sequence_length]

y_data = tf.constant([[1,1,1]])
# [batch_size, sequence_length, emb_dim ]
prediction1= tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2= tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

with tf.Session() as sess:
    sequence_loss1 = tf.contrib.seq2seq.sequence_loss(logits=prediction1, targets=y_data, weights=weights)
    sequence_loss2 = tf.contrib.seq2seq.sequence_loss(logits=prediction2, targets=y_data, weights=weights)
    
    sess.run(tf.global_variables_initializer())
    print("Loss1: ", sequence_loss1.eval(),
         "\nLoss2: ", sequence_loss2.eval())